In [1]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 15.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existin

In [ ]:
# Define the YAML content
data_yaml = """
train: /kaggle/input/human-bone-fractures-image-dataset-hbfmid/Human Bone Fractures Multi-modal Image Dataset (HBFMID)/Bone Fractures Detection/train/images
val: /kaggle/input/human-bone-fractures-image-dataset-hbfmid/Human Bone Fractures Multi-modal Image Dataset (HBFMID)/Bone Fractures Detection/valid/images
test: /kaggle/input/human-bone-fractures-image-dataset-hbfmid/Human Bone Fractures Multi-modal Image Dataset (HBFMID)/Bone Fractures Detection/test/images

nc: 12
names: ['Elbow', 'Finger', 'Forearm', 'Humerus', 'Shoulder', 'Femur', 'Shinbone', 'Knee', 'Hipbone', 'Wrist', 'Spinal cord', 'Some healthy bones']
"""

# Write it to a file
with open("/kaggle/working/data.yaml", "w") as f:
    f.write(data_yaml)


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # or yolov8s.pt, etc.
model.train(data="/kaggle/working/data.yaml", epochs=50, imgsz=640, batch=16)


In [2]:
import os
from PIL import Image

# Define dataset root
dataset_path = "/kaggle/input/human-bone-fractures-image-dataset-hbfmid/Human Bone Fractures Multi-modal Image Dataset (HBFMID)/Bone Fractures Detection"

# Folders to check
splits = ["train", "valid", "test"]

for split in splits:
    img_dir = os.path.join(dataset_path, split, "images")
    lbl_dir = os.path.join(dataset_path, split, "labels")
    
    img_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))]
    
    for img_file in img_files:
        lbl_file = img_file.rsplit('.', 1)[0] + ".txt"
        lbl_path = os.path.join(lbl_dir, lbl_file)
        img_path = os.path.join(img_dir, img_file)
        
        # Check if label file exists
        if not os.path.exists(lbl_path):
            print(f"Missing label: {lbl_path}")
        
        # Check if image can be opened
        try:
            with Image.open(img_path) as img:
                img.verify()
        except:
            print(f"Corrupt image: {img_path}")
        
        # Check class IDs
        try:
            with open(lbl_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    class_id = int(line.strip().split()[0])
                    if not (0 <= class_id < 12):
                        print(f"Invalid class ID in {lbl_path}: {class_id}")
        except:
            continue


In [3]:
data_yaml = """
train: {}/train/images
val: {}/valid/images
test: {}/test/images

nc: 10
names: ['Elbow', 'Finger', 'Forearm', 'Humerus', 'Shoulder', 'Femur', 'Shinbone', 'Knee', 'Hipbone', 'Wrist']
""".format(dataset_path, dataset_path, dataset_path)

with open("/kaggle/working/data.yaml", "w") as f:
    f.write(data_yaml)


In [4]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")  # use 's', 'm', or 'l' for larger models
model.train(data="/kaggle/working/data.yaml", epochs=200, imgsz=640, batch=16)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 124MB/s]


Ultralytics 8.3.127 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0

100%|██████████| 755k/755k [00:00<00:00, 22.8MB/s]


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.49it/s]


                   all        128        157       0.94      0.883      0.917      0.498
                 Elbow         15         15          1      0.687      0.828      0.502
                Finger         10         10      0.978        0.6      0.627      0.271
               Forearm          7          7      0.921          1      0.995      0.656
               Humerus          1          1      0.812          1      0.995      0.398
              Shoulder         30         30          1      0.904       0.96      0.427
                 Femur          7          7          1      0.952      0.995       0.59
              Shinbone          3          3      0.861          1      0.995      0.842
                  Knee          5          5       0.97          1      0.995      0.437
               Hipbone         53         65      0.855      0.725      0.784      0.327
                 Wrist         11         14          1      0.966      0.995      0.528


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ba6dc2c7510>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0

In [ ]:
metrics = model.val(split="test")  # uses test/images


In [ ]:
import os
from collections import Counter

# Dataset root path
dataset_root = "/kaggle/input/human-bone-fractures-image-dataset-hbfmid/Human Bone Fractures Multi-modal Image Dataset (HBFMID)/Bone Fractures Detection"
splits = ["train", "valid", "test"]

# Class names (you can add more if there are 11 or 12 classes)
class_names = ['Elbow', 'Finger', 'Forearm', 'Humerus', 'Shoulder', 'Femur',
               'Shinbone', 'Knee', 'Hipbone', 'Wrist']

# Store counters for each split
split_counters = {split: Counter() for split in splits}

# Process each split
for split in splits:
    label_path = os.path.join(dataset_root, split, "labels")
    
    for label_file in os.listdir(label_path):
        if label_file.endswith(".txt"):
            with open(os.path.join(label_path, label_file), "r") as f:
                for line in f:
                    if line.strip():
                        class_id = int(line.strip().split()[0])
                        split_counters[split][class_id] += 1

# Display counts
for split in splits:
    print(f"\n=== {split.upper()} SET ===")
    print(f"{'Class ID':<10} {'Class Name':<20} {'Count':<10}")
    print("="*45)
    for i in range(len(class_names)):
        print(f"{i:<10} {class_names[i]:<20} {split_counters[split][i]:<10}")
